In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
import zipfile
from functools import partial
from glob import glob

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d puneet6060/intel-image-classification

401 - Unauthorized


In [5]:
if not os.path.exists('./intel-image-classification/'):
    with zipfile.ZipFile("intel-image-classification.zip","r") as zip_ref:
        zip_ref.extractall("intel-image-classification")

In [6]:
ROOT_PATH = 'intel-image-classification'

config = {

    'TRAIN_PATH': os.path.join(ROOT_PATH, 'seg_train', 'seg_train'),
    'TEST_PATH': os.path.join(ROOT_PATH, 'seg_test', 'seg_test'),
    'VALID_PATH': os.path.join(ROOT_PATH, 'seg_pred', 'seg_pred'),
    'BATCH_SIZE' :16,
    'IMG_SIZE' : 224
}

classnames=['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

train_paths =  glob(os.path.join(config['TRAIN_PATH'], '*', '*'), recursive=True)
test_paths =  glob(os.path.join(config['TEST_PATH'], '*', '*'), recursive=True)

In [7]:
def get_labels(pathlist):
    labels = []
    for path in pathlist:
        labels.append(classnames.index(path.split(os.sep)[3]))
    labels = np.array(labels)
    return labels

In [8]:
train_labels = get_labels(train_paths)
test_labels = get_labels(test_paths)

In [9]:
import albumentations as A

def preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, size=[config['IMG_SIZE'], config['IMG_SIZE']])
    img = tf.cast(img, tf.float32)

    return img, label

def albumentations(img):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.VerticalFlip(p=0.5),
        A.CoarseDropout(p=0.5)
    ])
    
    transformed_image = transform(image=img)['image']
    return transformed_image

def apply_albumentations(img, label):
    aug_img = tf.numpy_function(func=albumentations, inp=[img], Tout=tf.float32)
    return aug_img, label

In [10]:
def create_dataset(images, labels):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels)).shuffle(len(images))
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).map(apply_albumentations).batch(config['BATCH_SIZE'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    return dataset

In [11]:
train_set = create_dataset(train_paths, train_labels)
test_set = create_dataset(test_paths, test_labels)

In [12]:
def make_model(jit=False, separable=False, mixed_precision=False, fuse_steps=1):
    if mixed_precision:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
    elif mixed_precision == False:
        tf.keras.mixed_precision.set_global_policy('float32')
    
    if separable:
        model = keras.models.Sequential([
          keras.layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same', input_shape=[224, 224, 3]),
          keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D((2, 2), (2, 2)),
          keras.layers.BatchNormalization(),

          keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
          keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
          keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
          keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.SeparableConv2D(512, 3, activation='relu', padding='same'),
          keras.layers.SeparableConv2D(512, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.GlobalAveragePooling2D(),    
          keras.layers.Dropout(0.3, seed=2),
          keras.layers.Dense(6, activation='softmax')
      ])
      else:
        model = keras.models.Sequential([
          keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=[224, 224, 3]),
          keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D((2, 2), (2, 2)),
          keras.layers.BatchNormalization(),

          keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
          keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
          keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
          keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
          keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
          keras.layers.MaxPooling2D(2, 2),
          keras.layers.BatchNormalization(),

          keras.layers.GlobalAveragePooling2D(),    
          keras.layers.Dropout(0.3, seed=2),
          keras.layers.Dense(6, activation='softmax')
      ])
  

      model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.Adam(),
                  jit_compile=jit,
                  steps_per_execution=fuse_steps,
                  metrics=['accuracy',
                           keras.metrics.SparseTopKCategoricalAccuracy(k=2)])
  
  return model

In [13]:
reducelr = keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.2)

In [14]:
plain = make_model()
plain.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     1

In [15]:
plain_history = plain.fit(train_set,
                    validation_data = test_set,
                    callbacks=[reducelr],
                    epochs = 20)

Epoch 1/20
877/877 [==============================] - 171s 189ms/step - loss: 1.4848 - accuracy: 0.3813 - sparse_top_k_categorical_accuracy: 0.6031 - val_loss: 1.6316 - val_accuracy: 0.4044 - val_sparse_top_k_categorical_accuracy: 0.6083 - lr: 0.0010
Epoch 2/20
877/877 [==============================] - 165s 188ms/step - loss: 1.2100 - accuracy: 0.4991 - sparse_top_k_categorical_accuracy: 0.7025 - val_loss: 1.8141 - val_accuracy: 0.3636 - val_sparse_top_k_categorical_accuracy: 0.5832 - lr: 0.0010
Epoch 3/20
877/877 [==============================] - 165s 188ms/step - loss: 1.1293 - accuracy: 0.5342 - sparse_top_k_categorical_accuracy: 0.7191 - val_loss: 1.1630 - val_accuracy: 0.5231 - val_sparse_top_k_categorical_accuracy: 0.7062 - lr: 0.0010
Epoch 4/20
877/877 [==============================] - 165s 188ms/step - loss: 1.0832 - accuracy: 0.5523 - sparse_top_k_categorical_accuracy: 0.7318 - val_loss: 1.0257 - val_accuracy: 0.5538 - val_sparse_top_k_categorical_accuracy: 0.7510 - lr: 0.0

In [16]:
separable = make_model(separable=True)
separable.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separable  (None, 224, 224, 64)     283       
 Conv2D)                                                         
                                                                 
 separable_conv2d_1 (Separab  (None, 224, 224, 64)     4736      
 leConv2D)                                                       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 112, 112, 64)     256       
 hNormalization)                                                 
                                                                 
 separable_conv2d_2 (Separab  (None, 112, 112, 128)   

In [17]:
separable_history = separable.fit(train_set,
                    validation_data = test_set,
                    callbacks=[reducelr],
                    epochs = 20)

Epoch 1/20
877/877 [==============================] - 253s 285ms/step - loss: 1.3489 - accuracy: 0.4065 - sparse_top_k_categorical_accuracy: 0.6484 - val_loss: 1.4346 - val_accuracy: 0.4281 - val_sparse_top_k_categorical_accuracy: 0.6624 - lr: 0.0010
Epoch 2/20
877/877 [==============================] - 251s 286ms/step - loss: 1.1321 - accuracy: 0.5315 - sparse_top_k_categorical_accuracy: 0.7264 - val_loss: 1.9086 - val_accuracy: 0.4047 - val_sparse_top_k_categorical_accuracy: 0.5979 - lr: 0.0010
Epoch 3/20
877/877 [==============================] - 249s 284ms/step - loss: 1.0105 - accuracy: 0.5864 - sparse_top_k_categorical_accuracy: 0.7647 - val_loss: 0.9210 - val_accuracy: 0.6240 - val_sparse_top_k_categorical_accuracy: 0.8055 - lr: 0.0010
Epoch 4/20
877/877 [==============================] - 249s 284ms/step - loss: 0.8997 - accuracy: 0.6418 - sparse_top_k_categorical_accuracy: 0.8277 - val_loss: 0.8957 - val_accuracy: 0.6661 - val_sparse_top_k_categorical_accuracy: 0.8469 - lr: 0.0

In [18]:
separable_jit = make_model(separable=True, jit=True)
separable_jit.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_10 (Separa  (None, 224, 224, 64)     283       
 bleConv2D)                                                      
                                                                 
 separable_conv2d_11 (Separa  (None, 224, 224, 64)     4736      
 bleConv2D)                                                      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 batch_normalization_10 (Bat  (None, 112, 112, 64)     256       
 chNormalization)                                                
                                                                 
 separable_conv2d_12 (Separa  (None, 112, 112, 128)   

In [19]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0


In [20]:
separable_jit_history = separable_jit.fit(train_set,
                    validation_data = test_set,
                    callbacks=[reducelr],
                    epochs = 20)

Epoch 1/20
877/877 [==============================] - 124s 102ms/step - loss: 1.3210 - accuracy: 0.4247 - sparse_top_k_categorical_accuracy: 0.6578 - val_loss: 1.2304 - val_accuracy: 0.4763 - val_sparse_top_k_categorical_accuracy: 0.7142 - lr: 0.0010
Epoch 2/20
877/877 [==============================] - 89s 101ms/step - loss: 1.1045 - accuracy: 0.5437 - sparse_top_k_categorical_accuracy: 0.7318 - val_loss: 1.0270 - val_accuracy: 0.5739 - val_sparse_top_k_categorical_accuracy: 0.7376 - lr: 0.0010
Epoch 3/20
877/877 [==============================] - 89s 101ms/step - loss: 1.0339 - accuracy: 0.5745 - sparse_top_k_categorical_accuracy: 0.7437 - val_loss: 1.1088 - val_accuracy: 0.5361 - val_sparse_top_k_categorical_accuracy: 0.7383 - lr: 0.0010
Epoch 4/20
877/877 [==============================] - 89s 101ms/step - loss: 0.9958 - accuracy: 0.5900 - sparse_top_k_categorical_accuracy: 0.7568 - val_loss: 0.9926 - val_accuracy: 0.5943 - val_sparse_top_k_categorical_accuracy: 0.7577 - lr: 0.0010

In [21]:
separable_jit_amp = make_model(separable=True, jit=True, mixed_precision=True)
separable_jit_amp.summary()

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 SUPER, compute capability 7.5
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_20 (Separa  (None, 224, 224, 64)     283       
 bleConv2D)                                                      
                                                                 
 separable_conv2d_21 (Separa  (None, 224, 224, 64)     4736      
 bleConv2D)                                                      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 batc

In [22]:
separable_jit_amp_history = separable_jit_amp.fit(train_set,
                    validation_data = test_set,
                    callbacks=[reducelr],
                    epochs = 20)

Epoch 1/20
877/877 [==============================] - 213s 77ms/step - loss: 1.3460 - accuracy: 0.4116 - sparse_top_k_categorical_accuracy: 0.6440 - val_loss: 1.1263 - val_accuracy: 0.5130 - val_sparse_top_k_categorical_accuracy: 0.7299 - lr: 0.0010
Epoch 2/20
877/877 [==============================] - 67s 77ms/step - loss: 1.1402 - accuracy: 0.5228 - sparse_top_k_categorical_accuracy: 0.7200 - val_loss: 1.0441 - val_accuracy: 0.5588 - val_sparse_top_k_categorical_accuracy: 0.7467 - lr: 0.0010
Epoch 3/20
877/877 [==============================] - 67s 77ms/step - loss: 1.0539 - accuracy: 0.5693 - sparse_top_k_categorical_accuracy: 0.7385 - val_loss: 1.2315 - val_accuracy: 0.5057 - val_sparse_top_k_categorical_accuracy: 0.7086 - lr: 0.0010
Epoch 4/20
877/877 [==============================] - 67s 77ms/step - loss: 0.9955 - accuracy: 0.5919 - sparse_top_k_categorical_accuracy: 0.7535 - val_loss: 0.9769 - val_accuracy: 0.5852 - val_sparse_top_k_categorical_accuracy: 0.7590 - lr: 0.0010
Epo

In [23]:
separable_jit_amp_stepf = make_model(separable=True, jit=True, mixed_precision=True, fuse_steps=3)
separable_jit_amp_stepf.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_30 (Separa  (None, 224, 224, 64)     283       
 bleConv2D)                                                      
                                                                 
 separable_conv2d_31 (Separa  (None, 224, 224, 64)     4736      
 bleConv2D)                                                      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 batch_normalization_20 (Bat  (None, 112, 112, 64)     256       
 chNormalization)                                                
                                                                 
 separable_conv2d_32 (Separa  (None, 112, 112, 128)   

In [24]:
separable_jit_amp_stepf_history = separable_jit_amp_stepf.fit(train_set,
                    validation_data = test_set,
                    callbacks=[reducelr],
                    epochs = 20)

Epoch 1/20
877/877 [==============================] - 74s 84ms/step - loss: 1.2912 - accuracy: 0.4457 - sparse_top_k_categorical_accuracy: 0.6690 - val_loss: 1.1696 - val_accuracy: 0.5221 - val_sparse_top_k_categorical_accuracy: 0.7059 - lr: 0.0010
Epoch 2/20
877/877 [==============================] - 66s 76ms/step - loss: 1.0694 - accuracy: 0.5564 - sparse_top_k_categorical_accuracy: 0.7449 - val_loss: 1.1186 - val_accuracy: 0.5328 - val_sparse_top_k_categorical_accuracy: 0.7373 - lr: 0.0010
Epoch 3/20
877/877 [==============================] - 66s 76ms/step - loss: 1.0271 - accuracy: 0.5766 - sparse_top_k_categorical_accuracy: 0.7500 - val_loss: 1.2138 - val_accuracy: 0.5321 - val_sparse_top_k_categorical_accuracy: 0.7330 - lr: 0.0010
Epoch 4/20
877/877 [==============================] - 66s 76ms/step - loss: 0.9821 - accuracy: 0.6042 - sparse_top_k_categorical_accuracy: 0.7592 - val_loss: 1.0868 - val_accuracy: 0.5665 - val_sparse_top_k_categorical_accuracy: 0.7320 - lr: 0.0010
Epoc

In [25]:
plain.evaluate(test_set)

187/187 [==============================] - 10s 51ms/step - loss: 0.8324 - accuracy: 0.6397 - sparse_top_k_categorical_accuracy: 0.7811


[0.8324045538902283, 0.6397058963775635, 0.7810828685760498]

In [26]:
separable.evaluate(test_set)

187/187 [==============================] - 7s 36ms/step - loss: 0.6741 - accuracy: 0.7433 - sparse_top_k_categorical_accuracy: 0.8877


[0.6740912199020386, 0.7433155179023743, 0.8877005577087402]

In [27]:
separable_jit.evaluate(test_set)

187/187 [==============================] - 4s 19ms/step - loss: 0.8206 - accuracy: 0.6835 - sparse_top_k_categorical_accuracy: 0.8693


[0.8206043243408203, 0.6834893226623535, 0.8693181872367859]

In [28]:
separable_jit_amp.evaluate(test_set)

187/187 [==============================] - 4s 22ms/step - loss: 0.7690 - accuracy: 0.7172 - sparse_top_k_categorical_accuracy: 0.8760


[0.7690488696098328, 0.7172459959983826, 0.8760026693344116]

In [29]:
separable_jit_amp_stepf.evaluate(test_set)

187/187 [==============================] - 4s 22ms/step - loss: 0.5980 - accuracy: 0.7714 - sparse_top_k_categorical_accuracy: 0.8934


[0.597995400428772, 0.7713903784751892, 0.8933823704719543]